In [3]:
from satellite_analysis.graphs import massrelation as massrelation
from satellite_analysis.massrelations import stellarmass_relations
import yt
import glob
import numpy as np

#first find all of the halomass catalogs

#add argparse and make the out_dir if it doesnt exits
output_dir = '/Users/user1/documents/GEN3/stellarmassrelation/images'
halomass_dirs = glob.glob('/Users/user1/documents/GEN3/stellarmassrelation/VELA*')

#now, find the earliest and latest snapshots for each catalog
earliest_snap = []
latest_snap = []
for directory in halomass_dirs:
    all_snaps = glob.glob('%s/*.ascii' % directory)
    all_snaps.sort()
    earliest_snap.append(all_snaps[0])
    latest_snap.append(all_snaps[-1])

earliest_snap = earliest_snap[0][-9:-6]
latest_snap = latest_snap[-1][-9:-6]

#now make the list of all possible snapshots from the earlies and latest snap
a_list = [int(earliest_snap) + 10*x for x in range((int(latest_snap) - int(earliest_snap))//10 + 1)]

#now now I set up a nested loop to iterate over each a in my a_list in each of my VELA_dir halomass folder
for a in a_list:
    print('Generating dataset for a: %s' % a)
    
    central_ids, satellite_ids = [], []
    Mvir, Mpeak = [], []
    central_stellarmass010rvir, satellite_stellarmass010rvir, = [], []
    central_darkmattermass010rvir, satellite_darkmattermass010rvir, = [], []
    central_ratio010rvir, satellite_ratio010rvir = [], []
    central_stellarmass015rvir, satellite_stellarmass015rvir, = [], []
    central_darkmattermass015rvir, satellite_darkmattermass015rvir, = [], []
    central_ratio015rvir, satellite_ratio015rvir = [], []
    central_stellarmass020rvir, satellite_stellarmass020rvir, = [], []
    central_darkmattermass020rvir, satellite_darkmattermass020rvir, = [], []
    central_ratio020rvir, satellite_ratio020rvir = [], []
    central_stellarmassrvir, satellite_stellarmassrvir, = [], []
    central_darkmattermassrvir, satellite_darkmattermassrvir, = [], []
    central_ratiorvir, satellite_ratiorvir = [], []
    
    for halomass_dir in halomass_dirs:
        print('Trying file: %s/halomass%s.ascii' % (halomass_dir, a))
        try:
            readfile = open('%s/halomass%s.ascii' % (halomass_dir, a))
            lines = readfile.readlines()
            halomass_lines = []
            for line in lines:
                halomass_lines.append(line.split())
            del halomass_lines[0]
            for data_lists in halomass_lines:
                #filter out the satellites (pid =/= -1) from the central galaxies they orbit (pid = -1)
                if float(data_lists[1]) == -1:
                    central_ids.append(float(data_lists[0]))
                    #need the Mvir for the centrals and the Mpeak for the satellites
                    Mvir.append(float(data_lists[2]))
                    central_stellarmass010rvir.append(float(data_lists[4]))
                    central_darkmattermass010rvir.append(float(data_lists[5]))
                    central_ratio010rvir.append(float(data_lists[4])/float(data_lists[2]))
                    central_stellarmass015rvir.append(float(data_lists[6]))
                    central_darkmattermass015rvir.append(float(data_lists[7]))
                    central_ratio015rvir.append(float(data_lists[6])/float(data_lists[2]))
                    central_stellarmass020rvir.append(float(data_lists[8]))
                    central_darkmattermass020rvir.append(float(data_lists[9]))
                    central_ratio020rvir.append(float(data_lists[8])/float(data_lists[2]))
                    central_stellarmassrvir.append(float(data_lists[10]))
                    central_darkmattermassrvir.append(float(data_lists[11]))
                    central_ratiorvir.append(float(data_lists[10])/float(data_lists[2]))
                else:
                    satellite_ids.append(float(data_lists[0]))
                    #need the Mvir for the centrals and the Mpeak for the satellites
                    Mpeak.append(float(data_lists[3]))
                    satellite_stellarmass010rvir.append(float(data_lists[4]))
                    satellite_darkmattermass010rvir.append(float(data_lists[5]))
                    satellite_ratio010rvir.append(float(data_lists[4])/float(data_lists[3]))
                    satellite_stellarmass015rvir.append(float(data_lists[6]))
                    satellite_darkmattermass015rvir.append(float(data_lists[7]))
                    satellite_ratio015rvir.append(float(data_lists[6])/float(data_lists[3]))
                    satellite_stellarmass020rvir.append(float(data_lists[8]))
                    satellite_darkmattermass020rvir.append(float(data_lists[9]))
                    satellite_ratio020rvir.append(float(data_lists[8])/float(data_lists[3]))
                    satellite_stellarmassrvir.append(float(data_lists[10]))
                    satellite_darkmattermassrvir.append(float(data_lists[11]))
                    satellite_ratiorvir.append(float(data_lists[10])/float(data_lists[3]))
            readfile.close()
        except FileNotFoundError:
            print('No file: %s/halomass%s.ascii' % (halomass_dir, a))
            
    #generate the Behroozi and Puebla relations to plot
    x = np.linspace(10**7.8, 10**14, 3000)
    Behroozirelation = stellarmass_relations.BehrooziWechslerConroy2013(a/1000, (1/(a/1000) - 1), x)
    Pueblarelation = stellarmass_relations.PueblaPrimack2017(a/1000, (1/(a/1000) - 1), x)
    #now use the extracted info from the current a to graph the relationships desired
    print('Generating Graphs')
    #generate graphs for .1, .15, .2, and 1.0 rvir for both only the centrals and satellites and then together
    massrelation.centralandsatellitegalaxypapercomparisons(Mvir,
                                                           central_stellarmass010rvir,
                                                           central_ratio010rvir,
                                                           Mpeak, 
                                                           satellite_stellarmass010rvir,
                                                           satellite_ratio010rvir,
                                                           Behroozirelation,
                                                           Pueblarelation, x,
                                                           output_dir, a, '010rvir')
    
    massrelation.centralandsatellitegalaxypapercomparisons(Mvir,
                                                           central_stellarmass015rvir,
                                                           central_ratio015rvir,
                                                           Mpeak, 
                                                           satellite_stellarmass015rvir,
                                                           satellite_ratio015rvir,
                                                           Behroozirelation,
                                                           Pueblarelation, x,
                                                           output_dir, a, '015rvir')   
    
    massrelation.centralandsatellitegalaxypapercomparisons(Mvir,
                                                           central_stellarmass020rvir,
                                                           central_ratio020rvir,
                                                           Mpeak, 
                                                           satellite_stellarmass020rvir,
                                                           satellite_ratio020rvir,
                                                           Behroozirelation,
                                                           Pueblarelation, x,
                                                           output_dir, a, '020rvir')
    
    massrelation.centralgalaxypapercomparisons(Mvir, central_stellarmass010rvir,
                                               central_ratio010rvir, Behroozirelation, Pueblarelation,
                                               x, output_dir, a, '010rvir')
    
    massrelation.centralgalaxypapercomparisons(Mvir, central_stellarmass015rvir,
                                               central_ratio015rvir, Behroozirelation, Pueblarelation,
                                               x, output_dir, a, '015rvir')
    
    massrelation.centralgalaxypapercomparisons(Mvir, central_stellarmass020rvir,
                                               central_ratio020rvir, Behroozirelation, Pueblarelation,
                                               x, output_dir, a, '020rvir')
       
    massrelation.satellitegalaxypapercomparisons(Mpeak, satellite_stellarmass010rvir,
                                                 satellite_ratio010rvir, Behroozirelation, Pueblarelation,
                                                 x, output_dir, a, '010rvir')
    
    massrelation.satellitegalaxypapercomparisons(Mpeak, satellite_stellarmass015rvir,
                                                 satellite_ratio015rvir, Behroozirelation, Pueblarelation,
                                                 x, output_dir, a, '015rvir')
    
    massrelation.satellitegalaxypapercomparisons(Mpeak, satellite_stellarmass020rvir,
                                                 satellite_ratio020rvir, Behroozirelation, Pueblarelation,
                                                 x, output_dir, a, '020rvir')
        
    

Generating dataset for a: 100
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA06/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA07/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA08/halomass100.ascii
No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA08/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA09/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA10/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA11/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA12/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA13/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA14/halomass100.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA15/halomass100.ascii


/Users/user1/anaconda3/lib/python3.6/site-packages/satellite_analysis/massrelations/stellarmass_relations.py:108: RuntimeWarning: overflow encountered in exp
  return(-np.log10((10**(alpha*x) + 1)) + delta*(np.log10(1 + np.exp(x)))**gamma / (1 + np.exp(10**-x)))
/Users/user1/anaconda3/lib/python3.6/site-packages/satellite_analysis/massrelations/stellarmass_relations.py:271: RuntimeWarning: overflow encountered in exp
  return(-np.log10(10**(-alpha*x) + 1) + delta*(np.log10(1 + np.exp(x)))**gamma / (1 + np.exp(10**-x)))


Generating Graphs
Generating dataset for a: 110
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA06/halomass110.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA07/halomass110.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA08/halomass110.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA09/halomass110.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA10/halomass110.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA11/halomass110.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA12/halomass110.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA13/halomass110.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA14/halomass110.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA15/halomass110.ascii
Generating Graphs
Generating dataset for a: 120
Trying file: /Users/user1/documents/GEN3/ste

Generating dataset for a: 200
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA06/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA07/halomass200.ascii
No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA07/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA08/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA09/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA10/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA11/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA12/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA13/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA14/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA15/halomass200.ascii
Generating Graphs
Generating

Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA15/halomass280.ascii
Generating Graphs
Generating dataset for a: 290
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA06/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA07/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA08/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA09/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA10/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA11/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA12/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA13/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA14/halomass290.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA15/halomass290.ascii
Genera

Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA15/halomass370.ascii
Generating Graphs
Generating dataset for a: 380
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA06/halomass380.ascii
No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA06/halomass380.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA07/halomass380.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA08/halomass380.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA09/halomass380.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA10/halomass380.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA11/halomass380.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA12/halomass380.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA13/halomass380.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA14/halomass380.ascii
Trying fil

No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA06/halomass460.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA07/halomass460.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA08/halomass460.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA09/halomass460.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA10/halomass460.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA11/halomass460.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA12/halomass460.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA13/halomass460.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA14/halomass460.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA15/halomass460.ascii
Generating Graphs
Generating dataset for a: 470
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA06/halomass470.ascii
No file: /

No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA06/halomass540.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA07/halomass540.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA08/halomass540.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA09/halomass540.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA10/halomass540.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA11/halomass540.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA12/halomass540.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA13/halomass540.ascii
No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA13/halomass540.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA14/halomass540.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA15/halomass540.ascii
Generating Graphs
Generating dataset for a: 550
Trying file: /

No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA08/halomass600.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA09/halomass600.ascii
No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA09/halomass600.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA10/halomass600.ascii
No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA10/halomass600.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA11/halomass600.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA12/halomass600.ascii
No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA12/halomass600.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA13/halomass600.ascii
No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA13/halomass600.ascii
Trying file: /Users/user1/documents/GEN3/stellarmassrelation/VELA14/halomass600.ascii
No file: /Users/user1/documents/GEN3/stellarmassrelation/VELA14/halomass60

In [2]:
stellar_mass_.1rvir_list = []

SyntaxError: invalid syntax (<ipython-input-2-13372a510f19>, line 1)

In [3]:
lista = [0.61076,  1235450, 0.00000,       -1,      8,       -1,       -1,       -1,  0, 1.14198e+10, 5.84000e+09,
 43.414000, 2.343000, 49.240000,  1, 0.10012, 43.290000, 11.87347, 12.04252, 12.20636, 79.120, 122.070, -201.250,
 5.019e+06, -7.777e+07, 1.444e+08, 0.01955,   601388,   601388,  1235450,     3067,   51,       -1,   627083,
 601439, 0.58955,  1234617, 2.34711, 6450600000, 5924100000, 5554800000, 4532100000, 3043600000, 1.77745, 4.23,
 0.02434, 0.97534, 0.81812, 1.05743, 0.69738, 0.62025, 0.96178, 0.83392, 0.93071, 0.58379, 0.50992, 0.7994,
 6937000000, 2356000000, 12.567, 5.84000e+09, 1.40300e+10, 43.290000, 77.680000, 0.13020, -2.525e-01, -2.525e-01,
 5.468e-02, 1.372e-01, 0.000e+00, 7.900e-11, -1.446e-11, 0.16033, 0.61076, 0.15002, 8.813e+09, 68.420, 77.290,
 0.60908, -0.25174, -1.00000,  1235450]

In [5]:
print(lista[61])
print(lista[10])

14030000000.0
5840000000.0
